This analysis explores the strategic shifts in Netflix's content library. By examining a dataset of over 8,000 titles (updated to 2021), I investigate how the platform balances movies versus TV shows, which global markets dominate production, and how content formats have evolved over time.

Research objectives
* Identifying which content categories lead the library.
* Ranking top-producing countries for both Movies and TV Shows.
* Analyzing trends in movie durations across different decades.
* Exploring recurring keywords in content descriptions.

To understand the structure of the dataset and verify column names, I will display the first few rows of the table.

In [17]:
-- Data preview
SELECT *
FROM "netflix_titles.csv"
LIMIT 5

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",None,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,None,None,None,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,None,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


1. The first step of this analysis is to confirm that the data has been imported correctly and to understand the overall scale of the Netflix library. I will check the total number of records and the distribution between Movies and TV Shows.

In [18]:
-- Check total number of titles
SELECT COUNT(*) AS total_titles 
FROM "netflix_titles.csv";

-- Check distribution by content type
SELECT type, COUNT(*) AS count_per_type
FROM "netflix_titles.csv"
GROUP BY type;

,type,count_per_type
0,Movie,6131
1,TV Show,2676


2. In this section, I analyze the most frequent genre combinations available on Netflix. This identifies the platform's content focus and popular categories.

In [19]:
-- Ranking the top 10 most frequent genre categories for Movies
SELECT 
    listed_in AS genre, 
    COUNT(*) AS total_count
FROM "netflix_titles.csv"
WHERE type = 'Movie'
GROUP BY listed_in
ORDER BY total_count DESC
LIMIT 10;

,genre,total_count
0,"Dramas, International Movies",362
1,Documentaries,359
2,Stand-Up Comedy,334
3,"Comedies, Dramas, International Movies",274
4,"Dramas, Independent Movies, International Movies",252
5,Children & Family Movies,215
6,"Children & Family Movies, Comedies",201
7,"Documentaries, International Movies",186
8,"Dramas, International Movies, Romantic Movies",180
9,"Comedies, International Movies",176


**Insight:** The analysis reveals that 'Documentaries' and 'Stand-Up Comedy' are among the most popular categories, suggesting Netflix's strong strategic focus on non-fiction and niche comedy content.

3. In this section, I identify the leading countries in content production. I will provide a Top 5 ranking for Movies and TV Shows separately to see if different regions dominate different formats. To achieve this, I will use a Common Table Expression (CTE) and the RANK() window function.

In [20]:
-- Using a CTE to aggregate content counts by type and country
WITH country_counts AS (
    SELECT 
        type, 
        country, 
        COUNT(*) AS total_titles
    FROM "netflix_titles.csv"
    WHERE country IS NOT NULL 
    GROUP BY type, country
),
-- Ranking countries within each content type (Movies/TV Shows)
ranked_countries AS (
    SELECT 
        type, 
        country, 
        total_titles,
        RANK() OVER (PARTITION BY type ORDER BY total_titles DESC) AS country_rank
    FROM country_counts
)
-- Filtering to show only the top 5 for each category
SELECT * FROM ranked_countries
WHERE country_rank <= 5
ORDER BY type, country_rank;

,type,country,total_titles,country_rank
0,Movie,United States,2058,1
1,Movie,India,893,2
2,Movie,United Kingdom,206,3
3,Movie,Canada,122,4
4,Movie,Spain,97,5
5,TV Show,United States,760,1
6,TV Show,United Kingdom,213,2
7,TV Show,Japan,169,3
8,TV Show,South Korea,158,4
9,TV Show,India,79,5


**Insight:** The United States leads in both formats, but the gap between the top producer and the rest of the world is narrower in the TV Shows category. Notably, India holds a strong second position for Movies, while the United Kingdom and Japan are more prominent in the TV Show rankings.

4. Has the average duration of movies changed over the years? This analysis helps determine if Netflix is shifting towards shorter, more "snackable" content or investing in longer cinematic experiences. To perform this, I will clean the 'duration' string and convert it to a numeric format for calculation.

In [21]:
-- Calculating average duration per year
SELECT 
    release_year, 
    ROUND(AVG(CAST(REPLACE(duration, ' min', '') AS INTEGER)), 2) AS avg_duration_min
FROM "netflix_titles.csv"
WHERE type = 'Movie' AND duration IS NOT NULL
GROUP BY release_year
ORDER BY release_year DESC;

,release_year,avg_duration_min
0,2021,96.44
1,2020,92.14
2,2019,93.47
3,2018,96.19
4,2017,95.54
...,...,...
68,1946,58.00
69,1945,51.33
70,1944,52.00
71,1943,62.67


**Insight:** The data suggests that movie lengths have stabilized in recent years. While there's a wide variety of runtimes, the average stays close to the 90-100 minute mark, showing that Netflix maintains a standard feature-length format for its movie library.

5. Beyond genres and metadata, the language used in plot summaries reveals the platform's thematic focus. In this final phase, I use conditional aggregation to quantify how often key themes—specifically "Love" and "Action"—are mentioned in Netflix's content descriptions. This provides a glimpse into the emotional and stylistic hooks used to attract viewers.

In [22]:
-- Counting occurrences of specific themes in descriptions
SELECT 
    SUM(CASE WHEN description LIKE '%love%' THEN 1 ELSE 0 END) AS love_count,
    SUM(CASE WHEN description LIKE '%action%' THEN 1 ELSE 0 END) AS action_count
FROM "netflix_titles.csv";

,love_count,action_count
0,690.0,107.0


**Final Conclusion:** The keyword analysis shows which themes are more prevalent in Netflix's marketing and storytelling. A higher count for one over the other indicates whether the library leans more towards romantic and emotional narratives or high-energy, genre-driven entertainment. This concludes the analysis, highlighting Netflix's diverse approach to catering to global audience preferences.